In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text4000 = pd.read_csv(".//Data/4000/4000-stories-VAD.csv")

In [3]:
text4000.head(1)

,url,length,title,text_no,author,story,valence,arousal,dominance,text_id,...,290,291,292,293,294,295,296,297,298,299
0,https://americanliterature.com/author/eleanor-...,15044,"Peace on Earth, Good-Will to Dogs",0,Eleanor Hallowell Abbott,"PART I\n\nIf you don't like Christmas stories,...",0.592896,0.397839,0.569567,0_eleanor-hallowell-abbott-peace-on-earth-good...,...,3.793141,3.837345,3.778354,-0.815515,-0.72044,-10.738245,-8.765683,0.875089,-10.176691,1.736791


In [4]:
text4000.keys()

Index(['url', 'length', 'title', 'text_no', 'author', 'story', 'valence',
       'arousal', 'dominance', 'text_id',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=317)

In [5]:
texts= list(text4000.story.values)

In [6]:
text_limiter=1000000 #10000 zeichen
joined_text = " ".join(texts)
limited_text = joined_text[:text_limiter]

In [7]:
# Tokenizer erstellen und Texte darauf anwenden
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(limited_text.lower())

In [8]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [9]:
context_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - context_words):
    input_words.append(tokens[i:i + context_words])
    next_words.append(tokens[i + context_words])

In [10]:
abc=10011
print(input_words[abc])
print(next_words[abc])

['heart', 'if', 'her', 'man', 'don', 't', 'seem', 'to', 'care', 'about']
it


In [11]:
X = np.zeros((len(input_words), context_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [12]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    Y[i, unique_token_index[next_words[i]]] = 1

In [13]:
# model = Sequential()
# model.add(LSTM(128, input_shape=(context_words, len(unique_tokens)), return_sequences=True))
# model.add(LSTM(128))
# model.add(Dense(len(unique_tokens)))
# model.add(Activation("softmax"))
model = load_model("model1000000w.tf")

In [14]:
# model.compile(loss="categorical_crossentropy", optimizer = RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, Y, batch_size=128, epochs=50, shuffle=True)

Epoch 1/50
1470/1470 [==============================] - 396s 267ms/step - loss: 2.7388 - accuracy: 0.5484
Epoch 2/50
1470/1470 [==============================] - 398s 271ms/step - loss: 2.7103 - accuracy: 0.5524
Epoch 3/50
1470/1470 [==============================] - 436s 297ms/step - loss: 2.7324 - accuracy: 0.5506
Epoch 4/50
1470/1470 [==============================] - 440s 299ms/step - loss: 2.7222 - accuracy: 0.5512
Epoch 5/50
1470/1470 [==============================] - 442s 300ms/step - loss: 2.7189 - accuracy: 0.5512
Epoch 6/50
1470/1470 [==============================] - 439s 299ms/step - loss: 2.7187 - accuracy: 0.5485
Epoch 7/50
1470/1470 [==============================] - 439s 299ms/step - loss: 2.7270 - accuracy: 0.5505
Epoch 8/50
1470/1470 [==============================] - 439s 299ms/step - loss: 2.7177 - accuracy: 0.5501
Epoch 9/50
1470/1470 [==============================] - 440s 299ms/step - loss: 2.7294 - accuracy: 0.5487
Epoch 10/50
1470/1470 [=======================

In [15]:
model.save("model1000000w.tf")

INFO:tensorflow:Assets written to: model1000000w.tf\assets


INFO:tensorflow:Assets written to: model1000000w.tf\assets


In [16]:
with open('MetaData/context_words.pickle', 'wb') as file:
    pickle.dump(context_words, file)
with open('MetaData/unique_tokens.pickle', 'wb') as file:
    pickle.dump(unique_tokens, file)
with open('MetaData/unique_token_index.pickle', 'wb') as file:
    pickle.dump(unique_token_index, file)   

In [17]:
len(joined_text)

78553398